In [12]:
import os
import psycopg2
import pandas as pd

In [13]:
# Connect to database and load data
db_params = {
    "host": os.getenv("LOCAL_HOST"),
    "user": os.getenv("LOCAL_USER"),
    "password": os.getenv("LOCAL_PW"),
    "port": os.getenv("LOCAL_PORT"),
    "dbname": os.getenv("LOCAL_DB")
}

try:
    conn = psycopg2.connect(**db_params)
    print("Database connection successful")
    sql_query = "SELECT * FROM dev.golden_table;"
    df = pd.read_sql_query(sql_query, conn)
    conn.close()
    print("Golden data loaded into DataFrame:")
    print(df.info())
except Exception as e:
    print(f"An error occurred: {e}")

Database connection successful


C:\Users\Leo\AppData\Local\Temp\ipykernel_27644\372339225.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, conn)


Golden data loaded into DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23038 entries, 0 to 23037
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   school_name             23038 non-null  object 
 1   school_type             23038 non-null  object 
 2   teachers_fte            22550 non-null  float64
 3   enrollment              22863 non-null  float64
 4   grade_eight_enrollment  21613 non-null  float64
 5   math_counts             22507 non-null  float64
 6   math_high_pct           22507 non-null  float64
 7   math_low_pct            19960 non-null  float64
 8   read_counts             22386 non-null  float64
 9   read_high_pct           22386 non-null  float64
 10  read_low_pct            19907 non-null  float64
 11  pct_hhi_150k_200k       23038 non-null  float64
 12  pct_hhi_220k_plus       23038 non-null  float64
 13  avg_natwalkind          23038 non-null  float64
 14  tot

## 1. Data Overview & Preprocessing
We will (a) preview the dataframe loaded as `df`, (b) select numeric features for clustering, (c) impute / drop missing values, (d) scale features, and (e) optionally apply dimensionality reduction (PCA) inside Optuna trials.

Metrics to compute later:
- Silhouette Score (higher better)
- Calinski-Harabasz Index (higher better)
- Davies-Bouldin Index (lower better)

We'll define reusable helper functions so both model families share logic.

In [8]:
# Basic shape & preview
print(f"DataFrame shape: {df.shape}")
display(df.head())

df_numeric = df.select_dtypes(include=['int64','float64']).copy()
print(f"Numeric subset shape: {df_numeric.shape}")
missing_pct = df_numeric.isna().mean().sort_values(ascending=False)
print("Missing value percentage (top 15):")
display(missing_pct.head(15))

DataFrame shape: (23038, 22)


,school_name,school_type,teachers_fte,enrollment,grade_eight_enrollment,math_counts,math_high_pct,math_low_pct,read_counts,read_high_pct,...,pct_hhi_220k_plus,avg_natwalkind,total_10_14,pct_10_14,pct_female_10_14,total_pop,hhi_150k_200k,hhi_220k_plus,schools_in_zip,dup_rank
0,1 LT Charles W. Whitcomb School,1,93.0,1077.0,370.0,71.0,9.0,6.0,71.0,5.0,...,5.03,12.317521,2081,0,0.0,41505,1896,2086,2,1
1,100 Academy of Engineering and Technology MS,1,NaN,147.0,47.0,5.0,49.0,0.0,7.0,49.0,...,0.75,12.120378,3841,0,0.0,47881,961,358,3,1
2,1R ELEMENTARY,1,12.0,191.0,25.0,9.0,79.0,80.0,9.0,59.0,...,1.34,8.287234,2008,0,0.0,25966,605,349,4,1
3,21st Century Charter Sch of Gary,1,96.0,1329.0,102.0,98.0,5.0,0.0,96.0,49.0,...,0.15,8.167196,528,0,0.0,6105,33,9,1,1
4,21st Century Cyber CS,1,72.0,1536.0,202.0,95.0,49.0,20.0,96.0,69.0,...,8.93,8.938298,2838,0,0.0,50510,2702,4511,4,1


Numeric subset shape: (23038, 20)
Missing value percentage (top 15):


read_low_pct              0.135906
math_low_pct              0.133605
grade_eight_enrollment    0.061854
read_counts               0.028301
read_high_pct             0.028301
math_counts               0.023049
math_high_pct             0.023049
teachers_fte              0.021182
enrollment                0.007596
pct_female_10_14          0.004384
schools_in_zip            0.000000
hhi_220k_plus             0.000000
hhi_150k_200k             0.000000
total_pop                 0.000000
pct_hhi_220k_plus         0.000000
dtype: float64

In [9]:
# Handle missing values: simple strategy (median). Could be enhanced later.
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import numpy as np

imputer = SimpleImputer(strategy='median')
scaled_features = imputer.fit_transform(df_numeric)
scaler = StandardScaler()
X = scaler.fit_transform(scaled_features)
print(f"Feature matrix ready. Shape: {X.shape}")

Feature matrix ready. Shape: (23038, 20)


## 2. Optimization Helpers (Optuna)
We define metric computation and a utility to optionally apply PCA inside each trial to reduce dimensionality (tuned as a hyperparameter).

In [10]:
# Ensure optuna is available (if running in an environment where pip install is allowed)
try:
    import optuna
except ImportError:
    import sys, subprocess
    !{sys.executable} -m pip install optuna --quiet
    import optuna

from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

# Cache original data for reuse
X_full = X  # already scaled


def prepare_features(trial, X_input):
    """Optionally apply PCA controlled by trial hyperparameters."""
    use_pca = trial.suggest_categorical('use_pca', [True, False])
    if use_pca:
        # limit components between 2 and min(50, n_features)
        max_comp = min(50, X_input.shape[1])
        n_components = trial.suggest_int('pca_components', 2, max_comp)
        pca = PCA(n_components=n_components, random_state=42)
        X_red = pca.fit_transform(X_input)
        return X_red, pca
    return X_input, None


def compute_cluster_metrics(X_data, labels):
    # Guard for metrics requiring >1 cluster and fewer than n_samples clusters
    unique_labels = set(labels)
    if len(unique_labels) <= 1 or len(unique_labels) >= len(labels):
        return {
            'silhouette': float('nan'),
            'calinski_harabasz': float('nan'),
            'davies_bouldin': float('nan')
        }
    return {
        'silhouette': silhouette_score(X_data, labels),
        'calinski_harabasz': calinski_harabasz_score(X_data, labels),
        'davies_bouldin': davies_bouldin_score(X_data, labels)
    }


def objective_wrapper(build_model_fn):
    def objective(trial):
        X_trial, pca_obj = prepare_features(trial, X_full)
        model = build_model_fn(trial)
        labels = model.fit_predict(X_trial)
        metrics = compute_cluster_metrics(X_trial, labels)
        # We'll optimize on silhouette (maximize)
        trial.set_user_attr('metrics', metrics)
        if pca_obj is not None:
            trial.set_user_attr('pca_components_actual', getattr(pca_obj, 'n_components_', None))
        return metrics['silhouette']
    return objective

print("Helper functions and metrics ready.")

Helper functions and metrics ready.


## 3. KMeans Optimization
We search hyperparameters: n_clusters, init method, algorithm, optional PCA usage & components.

In [ ]:
# Logging / output suppression utilities for cleaner Optuna runs
import contextlib, sys, io, logging

SUPPRESS_TRIAL_OUTPUT = True  # toggle this to see full trial logs

@contextlib.contextmanager
def silent_stdout(enabled=True):
    if not enabled:
        yield
        return
    new_target = io.StringIO()
    old_stdout = sys.stdout
    try:
        sys.stdout = new_target
        yield
    finally:
        sys.stdout = old_stdout

# Reduce Optuna logging level (shows only WARNING+)
if SUPPRESS_TRIAL_OUTPUT:
    optuna.logging.set_verbosity(optuna.logging.WARNING)
else:
    optuna.logging.set_verbosity(optuna.logging.INFO)

print("Optuna logging suppression active:" , SUPPRESS_TRIAL_OUTPUT)

#### Output Control
Trial-level Optuna output is suppressed. To see every trial, set `SUPPRESS_TRIAL_OUTPUT = False` in the previous cell and re-run the optimization cells (Sections 3 and 4). Only the best parameters and their metrics plus the top 5 results are displayed now.

In [ ]:
from sklearn.cluster import KMeans


def build_kmeans(trial):
    n_clusters = trial.suggest_int('kmeans_n_clusters', 2, 15)
    init = trial.suggest_categorical('kmeans_init', ['k-means++', 'random'])
    algorithm = trial.suggest_categorical('kmeans_algorithm', ['lloyd', 'elkan'])
    return KMeans(
        n_clusters=n_clusters,
        init=init,
        algorithm=algorithm,
        n_init='auto',
        random_state=42
    )

print("Running KMeans optimization (silence trials=", SUPPRESS_TRIAL_OUTPUT, ") ...")
with silent_stdout(SUPPRESS_TRIAL_OUTPUT):
    study_kmeans = optuna.create_study(direction='maximize', study_name='kmeans_clustering')
    study_kmeans.optimize(objective_wrapper(build_kmeans), n_trials=40, show_progress_bar=not SUPPRESS_TRIAL_OUTPUT)

best_k_params = study_kmeans.best_trial.params
best_k_metrics = study_kmeans.best_trial.user_attrs.get('metrics', {})
print("KMeans Best Params:")
print(best_k_params)
print("KMeans Best Metrics:")
print(best_k_metrics)
print(f"Best silhouette: {study_kmeans.best_value:.4f}")

# Build concise results DataFrame (top 5 by silhouette)
kmeans_results = []
for t in study_kmeans.trials:
    row = {**t.params}
    row.update(t.user_attrs.get('metrics', {}))
    kmeans_results.append(row)
import pandas as pd
kmeans_results_df = pd.DataFrame(kmeans_results)
if not kmeans_results_df.empty:
    display(kmeans_results_df.sort_values('silhouette', ascending=False).head())

[I 2025-10-05 07:53:34,470] A new study created in memory with name: kmeans_clustering


  0%|          | 0/40 [00:00<?, ?it/s]

[I 2025-10-05 07:53:41,286] Trial 0 finished with value: 0.1501475067415055 and parameters: {'use_pca': False, 'kmeans_n_clusters': 5, 'kmeans_init': 'random', 'kmeans_algorithm': 'elkan'}. Best is trial 0 with value: 0.1501475067415055.
[I 2025-10-05 07:53:47,856] Trial 1 finished with value: 0.17152391670496694 and parameters: {'use_pca': True, 'pca_components': 9, 'kmeans_n_clusters': 6, 'kmeans_init': 'random', 'kmeans_algorithm': 'lloyd'}. Best is trial 1 with value: 0.17152391670496694.
[I 2025-10-05 07:53:53,884] Trial 2 finished with value: 0.1350042164568471 and parameters: {'use_pca': False, 'kmeans_n_clusters': 8, 'kmeans_init': 'k-means++', 'kmeans_algorithm': 'elkan'}. Best is trial 1 with value: 0.17152391670496694.
[I 2025-10-05 07:54:00,320] Trial 3 finished with value: 0.1868049429867379 and parameters: {'use_pca': False, 'kmeans_n_clusters': 3, 'kmeans_init': 'k-means++', 'kmeans_algorithm': 'lloyd'}. Best is trial 3 with value: 0.1868049429867379.
[I 2025-10-05 07:54

KeyboardInterrupt: 

## 4. Gaussian Mixture (GMM) Optimization
We tune: n_components, covariance_type, reg_covar, and optional PCA usage.

In [ ]:
from sklearn.mixture import GaussianMixture

print("Running GMM optimization (silence trials=", SUPPRESS_TRIAL_OUTPUT, ") ...")

def build_gmm(trial):
    n_components = trial.suggest_int('gmm_n_components', 2, 15)
    covariance_type = trial.suggest_categorical('gmm_covariance_type', ['full', 'tied', 'diag', 'spherical'])
    reg_covar = trial.suggest_float('gmm_reg_covar', 1e-6, 1e-2, log=True)
    return GaussianMixture(
        n_components=n_components,
        covariance_type=covariance_type,
        reg_covar=reg_covar,
        random_state=42
    )

with silent_stdout(SUPPRESS_TRIAL_OUTPUT):
    study_gmm = optuna.create_study(direction='maximize', study_name='gmm_clustering')
    study_gmm.optimize(objective_wrapper(build_gmm), n_trials=40, show_progress_bar=not SUPPRESS_TRIAL_OUTPUT)

best_g_params = study_gmm.best_trial.params
best_g_metrics = study_gmm.best_trial.user_attrs.get('metrics', {})
print("GMM Best Params:")
print(best_g_params)
print("GMM Best Metrics:")
print(best_g_metrics)
print(f"Best silhouette: {study_gmm.best_value:.4f}")

gmm_results = []
for t in study_gmm.trials:
    row = {**t.params}
    row.update(t.user_attrs.get('metrics', {}))
    gmm_results.append(row)
import pandas as pd
gmm_results_df = pd.DataFrame(gmm_results)
if not gmm_results_df.empty:
    display(gmm_results_df.sort_values('silhouette', ascending=False).head())

## 5. Visualization (2D PCA Projections)
We project the full standardized feature matrix to 2 principal components (outside of optimization) for consistent side-by-side cluster plots.

In [ ]:
import plotly.express as px

# Refit best models on (possibly PCA-transformed) feature space chosen by best trial
best_kmeans_params = study_kmeans.best_trial.params
best_gmm_params = study_gmm.best_trial.params

# Build best models again without PCA reduction (for consistent plotting base); PCA only for 2D viz
kmeans_best = KMeans(
    n_clusters=best_kmeans_params['kmeans_n_clusters'],
    init=best_kmeans_params['kmeans_init'],
    algorithm=best_kmeans_params['kmeans_algorithm'],
    n_init='auto',
    random_state=42
).fit(X_full)

gmm_best = GaussianMixture(
    n_components=best_gmm_params['gmm_n_components'],
    covariance_type=best_gmm_params['gmm_covariance_type'],
    reg_covar=best_gmm_params['gmm_reg_covar'],
    random_state=42
).fit(X_full)

labels_kmeans = kmeans_best.predict(X_full)
labels_gmm = gmm_best.predict(X_full)

# PCA for viz only
pca_viz = PCA(n_components=2, random_state=42)
X_2d = pca_viz.fit_transform(X_full)
plot_df = pd.DataFrame({
    'PC1': X_2d[:,0],
    'PC2': X_2d[:,1],
    'KMeans_Cluster': labels_kmeans.astype(str),
    'GMM_Cluster': labels_gmm.astype(str)
})

fig1 = px.scatter(plot_df, x='PC1', y='PC2', color='KMeans_Cluster', title='KMeans Clusters (PCA 2D)')
fig1.show()
fig2 = px.scatter(plot_df, x='PC1', y='PC2', color='GMM_Cluster', title='GMM Clusters (PCA 2D)')
fig2.show()

print("Visualization complete.")

## 6. Cluster Profiling & Comparison
Generate aggregate statistics per cluster for both algorithms and compare metrics side-by-side.

In [ ]:
# Attach cluster labels back to original numeric df
profile_df = df_numeric.copy()
profile_df['kmeans_cluster'] = labels_kmeans
profile_df['gmm_cluster'] = labels_gmm

kmeans_profile = profile_df.groupby('kmeans_cluster').agg(['mean','median','count'])
gmm_profile = profile_df.groupby('gmm_cluster').agg(['mean','median','count'])

print("KMeans cluster profile (first 10 features):")
display(kmeans_profile.iloc[:, :30])  # limit columns for display
print("GMM cluster profile (first 10 features):")
display(gmm_profile.iloc[:, :30])

# Consolidate top metrics
comparison_df = pd.DataFrame({
    'model': ['KMeans', 'GMM'],
    'best_silhouette': [study_kmeans.best_value, study_gmm.best_value],
    'best_params': [study_kmeans.best_trial.params, study_gmm.best_trial.params]
})

# Fetch corresponding Calinski-Harabasz and Davies-Bouldin from best trials
comparison_df['calinski_harabasz'] = [
    study_kmeans.best_trial.user_attrs['metrics']['calinski_harabasz'],
    study_gmm.best_trial.user_attrs['metrics']['calinski_harabasz']
]
comparison_df['davies_bouldin'] = [
    study_kmeans.best_trial.user_attrs['metrics']['davies_bouldin'],
    study_gmm.best_trial.user_attrs['metrics']['davies_bouldin']
]

print("Model comparison metrics:")
display(comparison_df)

# Simple relative ranking summary
ranking = comparison_df[['model','best_silhouette','calinski_harabasz','davies_bouldin']].copy()
ranking['silhouette_rank'] = ranking['best_silhouette'].rank(ascending=False)
ranking['ch_rank'] = ranking['calinski_harabasz'].rank(ascending=False)
ranking['db_rank'] = ranking['davies_bouldin'].rank(ascending=True)
ranking['avg_rank'] = ranking[['silhouette_rank','ch_rank','db_rank']].mean(axis=1)
print("Ranking summary:")
display(ranking.sort_values('avg_rank'))

## 7. Notes & Next Steps
Potential enhancements:
- Add DBSCAN / HDBSCAN for density-based perspective.
- Use feature selection or domain-driven grouping before clustering.
- Evaluate stability across bootstrap samples.
- Store cluster assignments back to database for downstream analytics.

## 8. Persist Best Models
Save best KMeans and GMM models, hyperparameters, and metrics into `src/unsupervised/` for reuse.

In [ ]:
import json, joblib, pathlib, datetime

# Directory to save models
save_dir = pathlib.Path('src') / 'unsupervised'
save_dir.mkdir(parents=True, exist_ok=True)

# Recreate best models if not already in memory (guard if cell order differs)
if 'kmeans_best' not in globals():
    from sklearn.cluster import KMeans
    kmeans_best = KMeans(
        n_clusters=study_kmeans.best_trial.params['kmeans_n_clusters'],
        init=study_kmeans.best_trial.params['kmeans_init'],
        algorithm=study_kmeans.best_trial.params['kmeans_algorithm'],
        n_init='auto',
        random_state=42
    ).fit(X_full)

# GMM study might not yet be executed; wrap in try
try:
    if 'study_gmm' in globals():
        from sklearn.mixture import GaussianMixture
        if 'gmm_best' not in globals():
            gmm_best = GaussianMixture(
                n_components=study_gmm.best_trial.params['gmm_n_components'],
                covariance_type=study_gmm.best_trial.params['gmm_covariance_type'],
                reg_covar=study_gmm.best_trial.params['gmm_reg_covar'],
                random_state=42
            ).fit(X_full)
    else:
        gmm_best = None
except Exception as e:
    print(f"Could not rebuild GMM best model: {e}")
    gmm_best = None

# Metadata assembly
now_ts = datetime.datetime.utcnow().isoformat() + 'Z'
meta = {
    'timestamp_utc': now_ts,
    'kmeans': {
        'params': study_kmeans.best_trial.params,
        'metrics': study_kmeans.best_trial.user_attrs.get('metrics', {})
    },
}
if 'study_gmm' in globals():
    meta['gmm'] = {
        'params': study_gmm.best_trial.params,
        'metrics': study_gmm.best_trial.user_attrs.get('metrics', {})
    }
else:
    meta['gmm'] = None

# Save models & meta
joblib.dump(kmeans_best, save_dir / 'kmeans_best_model.joblib')
if 'gmm_best' in globals() and gmm_best is not None:
    joblib.dump(gmm_best, save_dir / 'gmm_best_model.joblib')
with open(save_dir / 'unsupervised_models_metadata.json', 'w') as f:
    json.dump(meta, f, indent=2)

print(f"Saved artifacts to: {save_dir.resolve()}")
print("Files:")
for p in save_dir.glob('*model.joblib'):
    print(' -', p.name)
print(' - unsupervised_models_metadata.json')

### Loading Later
```python
import joblib, json
kmeans = joblib.load('src/unsupervised/kmeans_best_model.joblib')
try:
    gmm = joblib.load('src/unsupervised/gmm_best_model.joblib')
except FileNotFoundError:
    gmm = None
with open('src/unsupervised/unsupervised_models_metadata.json') as f:
    meta = json.load(f)
```
The metadata file contains hyperparameters and validation metrics captured at save time.